# Time Series Analysis: "The Final Project"

`End? No, the journey doesn't end here. Death is just another path. One that we all must take.
-J.R.R. Tolkien, The Return of the King`

---

## Libraries

In [1]:
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import statsmodels.api as sm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from pandas.plotting import register_matplotlib_converters
from IPython.display import display
# from tsa_functions import *

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=4)
pd.set_option('precision', 4)


In [2]:
from tsa_tools import *

ImportError: cannot import name 'TimeseriesGenerator' from partially initialized module 'tsa_tools' (most likely due to a circular import) (c:\Users\aamorado\Documents\tsa2021-m5\tsa_tools.py)

---

## M5 Forecasting

For this "final project", we will be forecasting the <b><u>level 9</b></u> series (unit sales of all products, aggregated for each store and department).

Load `sales_train_evaluation.csv` and use observations from `d_1 to d_1913` for training and `d_1914 to d_1941` for testing.

In [ ]:
df_calendar = pd.read_csv('../data/m5/calendar.csv')
df_sales = pd.read_csv('../data/m5/sales_train_evaluation.csv')
df_weights = pd.read_csv('../data/m5/weights_validation.csv')
# display(df_calendar, df_sales, df_weights)

In [ ]:
train_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[:-28])
test_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[-28:])
# display(train_df, test_df)

In [ ]:
levels = {
    1: None,
    2: "state_id",
    3: "store_id",
    4: "cat_id",
    5: "dept_id",
    6: ["state_id", "cat_id"],
    7: ["state_id", "dept_id"],
    8: ["store_id", "cat_id"],
    9: ["store_id", "dept_id"],
    10: "item_id",
    11: ["state_id", "item_id"],
    12: ["store_id", "item_id"]
}


---

## Part 1. Baseline Methods (10 pts.)

### Q1. (10 pts.)

Extract all level 9 series from the dataset.

For each series, generate a 28-step forecast using the methods enumerated below and calculate the `RMSSE` against the test set:

1. `Naive`


2. `Seasonal Naive`


3. `SES`


4. `Holt's Linear`


5. `Additive Holt-Winters`

Summarize the metrics in a dataframe and print it.

In [ ]:
methods = {
    "Naive": BaseFuncModel(naivef),
    "Seasonal Naive": BaseFuncModel(snaivef, m=7),
    "SES": StatsModelsWrapper(ETSModel, trend=None, seasonal=None),
    "Holt's Linear": StatsModelsWrapper(ETSModel, trend='add', seasonal=None),
    "Additive Holt-Winters": StatsModelsWrapper(
        ETSModel, seasonal_periods=7, trend='add', seasonal='add'),
}

train_df_9 = timeSeriesFiltering(
    train_df.sum(axis='columns', level=levels[9]), lower=10)
test_df_9 = test_df.sum(axis='columns', level=levels[9])
weights_df_9 = (df_weights
                .loc[df_weights['Level_id'] == 'Level9']
                .set_index(['Agg_Level_1', 'Agg_Level_2'])[['Weight']])

In [ ]:
res = {}
for method, model in methods.items():
    forecast_df_9 = pd.DataFrame(
        {label: model.fit(content).forecast(28)
        for label, content in train_df_9.items()}
        )
    res[method] = rateMyForecast(train_df_9, test_df_9, forecast_df_9)['RMSSE']

In [ ]:
pd.set_option('display.max_rows', None)
df_res_9_base = pd.DataFrame(res)
df_res_9_base.index = pd.MultiIndex.from_tuples(df_res_9_base.index)
df_res_9_base

Naive  Seasonal Naive     SES  Holt's Linear  \
CA_1 HOBBIES_1    1.4583          0.7619  0.8819         0.8810   
     HOBBIES_2    1.9340          1.1429  0.8831         0.8748   
     HOUSEHOLD_1  2.1052          0.5172  1.1514         1.1636   
     HOUSEHOLD_2  2.2997          0.5228  1.2208         1.1789   
     FOODS_1      0.9319          0.7312  0.9179         0.9362   
     FOODS_2      2.0535          0.8269  2.0534         2.3136   
     FOODS_3      1.7113          0.4944  1.0825         1.0185   
CA_2 HOBBIES_1    1.2053          0.7055  1.1179         1.1180   
     HOBBIES_2    1.3546          1.4893  1.3887         1.3790   
     HOUSEHOLD_1  2.0287          0.6373  1.3960         1.3853   
     HOUSEHOLD_2  1.9340          0.5920  1.4625         1.4534   
     FOODS_1      1.2557          1.1311  1.3875         1.3786   
     FOODS_2      4.2936          1.4906  2.3041         2.2415   
     FOODS_3      2.5830          0.6046  1.4502         1.4114   
CA_3 HOBBIES_1    0.9370          0.6647  0.6999         0.7018   
     HOBBIES_2    1.1984          1.6617  1.5530         1.5433   
     HOUSEHOLD_1  2.6217          0.7564  1.1273         1.1536   
     HOUSEHOLD_2  2.1039          0.8808  1.4258         1.4169   
     FOODS_1      1.1674          1.5874  1.3966         1.3892   
     FOODS_2      1.8816          0.7316  1.7271         1.7429   
     FOODS_3      1.5710          0.4353  0.8908         1.5877   
CA_4 HOBBIES_1    1.4556          1.1008  0.9956         0.9902   
     HOBBIES_2    2.1142          1.8563  1.8756         1.8751   
     HOUSEHOLD_1  3.4088          1.4737  1.2447         1.2169   
     HOUSEHOLD_2  1.3534          0.9567  1.0520         1.0386   
     FOODS_1      1.0167          1.5664  1.3857         1.3656   
     FOODS_2      1.2989          0.7998  1.1609         1.1533   
     FOODS_3      1.4144          0.6385  0.9469         0.9431   
TX_1 HOBBIES_1    1.6331          0.9169  0.9061         0.9046   
     HOBBIES_2    1.8823          1.6475  1.4296         1.4159   
     HOUSEHOLD_1  1.8399          0.8196  1.1676         1.1608   
     HOUSEHOLD_2  1.2045          1.2523  1.0488         1.0328   
     FOODS_1      1.3344          1.7189  1.5566         1.5460   
     FOODS_2      1.1906          0.9520  0.8562         0.8561   
     FOODS_3      1.1678          0.8416  0.9161         0.8855   
TX_2 HOBBIES_1    0.7979          0.8792  0.8789         0.8742   
     HOBBIES_2    0.7499          1.2548  0.9735         0.9685   
     HOUSEHOLD_1  0.8662          0.8529  0.8683         0.8712   
     HOUSEHOLD_2  1.3706          0.6559  0.9314         0.9346   
     FOODS_1      1.4774          1.2816  1.2919         1.2839   
     FOODS_2      0.9424          0.6834  0.7532         0.7719   
     FOODS_3      0.8228          0.6226  0.7499         0.7481   
TX_3 HOBBIES_1    1.5309          0.9025  0.9404         0.9370   
     HOBBIES_2    1.4339          1.7212  1.1186         1.1152   
     HOUSEHOLD_1  1.0738          1.0794  1.0248         1.0064   
     HOUSEHOLD_2  0.8605          1.3137  1.0128         0.9889   
     FOODS_1      2.7332          2.2206  2.1542         2.1356   
     FOODS_2      1.3682          1.4195  1.4254         1.4277   
     FOODS_3      0.9672          1.1046  1.0692         1.0493   
WI_1 HOBBIES_1    1.3685          0.5084  0.7533         0.7533   
     HOBBIES_2    1.2701          1.5892  1.2413         1.2333   
     HOUSEHOLD_1  1.7201          0.6488  1.0982         1.0990   
     HOUSEHOLD_2  1.7301          0.7509  1.1014         1.0997   
     FOODS_1      0.9843          0.8454  1.0438         1.0400   
     FOODS_2      2.1064          1.2493  1.6349         1.6291   
     FOODS_3      1.6655          0.5440  1.1554         1.1465   
WI_2 HOBBIES_1    1.3672          0.8269  0.7654         0.7546   
     HOBBIES_2    1.1960          1.5164  1.2308         1.2258   
     HOUSEHOLD_1  1.3142          1.0984  1.3773         1.3079   

---

## Part 2. LightGBM (30 pts.)

### Q2. (10 pts.)

For all series, use an un-tuned `LightGBM` with 56-day lookback that uses a one-step recursive forecasting strategy to generate a 28-step forecast.

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [ ]:
model = RecursiveRegressor(
    lgb.LGBMRegressor(random_state=1, w=56, h=28))  # Model: recursive-forecasting
pred = {}

for col in train_df_9:
    model.fit(None, train_df_9[col])
    pred[col] = model.predict(train_df_9[col].iloc[-56:]).squeeze()

NameError: name 'TimeseriesGenerator' is not defined

In [ ]:
lbm = lgb.LGBMRegressor(w=4)
lbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'w': 4}

### Q3. (10 pts.)

For all series, use an un-tuned `LightGBM` with 56-day lookback that uses a direct forecasting strategy to generate a 28-step forecast.

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [ ]:
model = MultiOutputRegressor(
    lgb.LGBMRegressor(random_state=1),
    n_jobs=1)  # Model: direct-forecasting
pred = {}

for col in train_df_9:
    X_train, X_test, y_train, y_test = TimeseriesGenerator(
        X=train_df_9[col],
        y=test_df_9[col],
        w=56,
        h=28)
    model.fit(X_train, y_train)
    pred[col] = model.predict(X_test).squeeze()

In [ ]:
df_pred_9_morlgb = pd.DataFrame(pred, index=test_df_9.index)

df_res_9_morlgb = rateMyForecast(
    train_df_9, test_df_9, df_pred_9_morlgb)['RMSSE']
res["MultiOutputRegressor(LGBMRegressor)"] = df_res_9_morlgb

df_res_9_morlgb.index = pd.MultiIndex.from_tuples(df_res_9_morlgb.index)
df_res_9_morlgb.to_frame()

RMSSE
CA_1 HOBBIES_1    0.7047
     HOBBIES_2    0.7176
     HOUSEHOLD_1  0.3733
     HOUSEHOLD_2  0.6859
     FOODS_1      0.6755
     FOODS_2      0.7744
     FOODS_3      0.5715
CA_2 HOBBIES_1    0.6795
     HOBBIES_2    1.5369
     HOUSEHOLD_1  0.5772
     HOUSEHOLD_2  0.8787
     FOODS_1      1.2751
     FOODS_2      1.6306
     FOODS_3      0.7208
CA_3 HOBBIES_1    0.5596
     HOBBIES_2    1.3369
     HOUSEHOLD_1  0.5958
     HOUSEHOLD_2  1.0737
     FOODS_1      1.2296
     FOODS_2      0.5397
     FOODS_3      0.5109
CA_4 HOBBIES_1    1.0998
     HOBBIES_2    1.5212
     HOUSEHOLD_1  1.2565
     HOUSEHOLD_2  1.2729
     FOODS_1      1.5578
     FOODS_2      0.8750
     FOODS_3      0.8486
TX_1 HOBBIES_1    0.7625
     HOBBIES_2    1.3809
     HOUSEHOLD_1  0.8403
     HOUSEHOLD_2  1.2187
     FOODS_1      1.4830
     FOODS_2      0.6484
     FOODS_3      0.7669
TX_2 HOBBIES_1    0.8344
     HOBBIES_2    1.0724
     HOUSEHOLD_1  0.7089
     HOUSEHOLD_2  0.5160
     FOODS_1      1.0602
     FOODS_2      0.6406
     FOODS_3      0.5537
TX_3 HOBBIES_1    0.7057
     HOBBIES_2    1.3886
     HOUSEHOLD_1  0.8919
     HOUSEHOLD_2  1.1623
     FOODS_1      1.7153
     FOODS_2      1.0512
     FOODS_3      0.9709
WI_1 HOBBIES_1    0.4981
     HOBBIES_2    1.0785
     HOUSEHOLD_1  0.5807
     HOUSEHOLD_2  0.7464
     FOODS_1      0.9986
     FOODS_2      0.9520
     FOODS_3      0.7062
WI_2 HOBBIES_1    0.7246
     HOBBIES_2    1.2962
     HOUSEHOLD_1  1.0141
     HOUSEHOLD_2  0.8986
     FOODS_1      2.5739
     FOODS_2      1.4896
     FOODS_3      0.8641
WI_3 HOBBIES_1    0.8809
     HOBBIES_2    1.1314
     HOUSEHOLD_1  0.7228
     HOUSEHOLD_2  0.8550
     FOODS_1      1.6744
     FOODS_2      0.8274
     FOODS_3      0.5258

### Q4. (10 pts.)

For all series, generate a 28-step forecast by combining the forecasts generated by the models in Q2 and Q3 (i.e. simple averaging).

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [ ]:
# Your code here

---

## Part 3. WRMSSE (10 pts.)

### Q5.  (10 pts.)

Calculate the `WRMSSE` for the all the methods described above. The weights can be found in `weights_validation.csv`.

For reference, the M5 benchmarks have the following `WRMSSE` scores at level 9:

- `Naive` = <b>1.764</b>


- `S.Naive` = <b>0.888</b>


- `ES_bu` = <b>0.728</b>

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [ ]:
(df_res_9_base.rename_axis(['Agg_Level_1', 'Agg_Level_2'])
 .multiply(weights_df_9.squeeze(), axis=0).sum())

Naive                    1.6287
Seasonal Naive           0.9300
SES                      1.2075
Holt's Linear            1.2367
Additive Holt-Winters    0.8549
dtype: float64

---

## Part 4. Middle-Out Method (30 pts.)

### Q6. Bottom-Up (15 pts.)

Using your forecasts from the best performing method in Q5, use the bottom-up method described in [FPP3](https://otexts.com/fpp3/single-level.html) to generate forecasts for levels 1 to 8.

Calculate the `WRMSSE` for levels 1 to 8 against the test set, then summarize the metrics in a dataframe and print it.

For reference, you can find the benchmark `WRMSSE` scores in the `The M5 Accuracy competition: Results, findings and conclusions` paper.

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [ ]:
# Your code here

### Q7. Top-Down  (15 pts.)

Using your forecasts from the best performing method in Q5, use the top-down method with `average historical proportions` described in [FPP3](https://otexts.com/fpp3/single-level.html) to generate forecasts for levels 10 to 12.

Calculate the `WRMSSE` for levels 10 to 12  against the test set, then summarize the metrics in a dataframe and print it.

For reference, you can find the benchmark `WRMSSE` scores in the `The M5 Accuracy competition: Results, findings and conclusions` paper.

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [ ]:
# Your code here